In [1]:
import numpy as np
import pandas as pd

import os
from os import walk
from os import path
from os import listdir
from os.path import isfile, join
from shutil import copyfile

In [2]:
mypath = './job_outputs/logs/'
filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]


In [3]:
from collections import defaultdict
job_id_dict = defaultdict(dict)
solution_dict = defaultdict(dict)

In [4]:
for file in filenames:
    with open(path.join(mypath,file), 'r') as f:
        if os.stat(path.join(mypath,file)).st_size == 0: continue # check if file is empty
        if file[0]=='r': # reedbush job output file
            x = f.readlines()
            seed_line = x[0] # line containing seed
            tag_line = x[-2] # line containing experiment tag
            frame_line = x[-1] # line containing no. of frames required to solve

            if seed_line.split()[0]=="SEED:": # check if correct line
                seed = seed_line.split()[1] # get seed

            if frame_line.split()[0]=="Solved": # check if solved
                frames = frame_line.split()[2] # get no. of frames required to solve
            else:
                frames ="FAILED" # if not solved
                tag_line = x[-1]

            experiment_name = tag_line.split()[1][0:-23]
            job_id = file[-7:]    
            job_id_dict.update({(experiment_name,seed):job_id})
            solution_dict.update({(experiment_name,seed):frames})

            # copy the file to an organized directory
            cur_folder = mypath#os.getcwd()
            log_folder = os.path.join(cur_folder,experiment_name)
            if not os.path.exists(log_folder):
                os.makedirs(log_folder)
            log_file = job_id + '_' + tag_line.split()[1] + '.txt'
            log_file = os.path.join(log_folder, log_file)

            copyfile(path.join(mypath,file), log_file)
        
        elif file[0]=='i':
            x = f.readlines()
            for line in range(len(x)):
                if x[line] == '\n': 
                    continue
                elif x[line].split()[0] =='SEED:':
                    seed_line = x[line]
                elif x[line].split()[0] == 'TAG:':
                    tag_line = x[line]
                elif x[line].split()[0] == 'Solved':
                    frame_line = x[line]
                else:
                    frame_line = None
    #             seed_line = x[0] # line containing seed
    #             tag_line = x[-2] # line containing experiment tag
    #             frame_line = x[-1] # line containing no. of frames required to solve

            if seed_line.split()[0]=="SEED:": # check if correct line
                seed = seed_line.split()[1] # get seed
            else:
                print("BAD FILE:", file)
                continue

            if frame_line is not None: # check if solved
                frames = frame_line.split()[2] # get no. of frames required to solve
            else:
                frames ="FAILED" # if not solved
#                 tag_line = x[-1]

            experiment_name = tag_line.split()[1][0:-23]
            job_id = file[-6:]    
            job_id_dict.update({(experiment_name,seed):job_id})
            solution_dict.update({(experiment_name,seed):frames})

            # copy the file to an organized directory
            cur_folder = mypath#os.getcwd()
            log_folder = os.path.join(cur_folder,experiment_name)
            if not os.path.exists(log_folder):
                os.makedirs(log_folder)
            log_file = job_id + '_' + tag_line.split()[1] + '.txt'
            log_file = os.path.join(log_folder, log_file)

            copyfile(path.join(mypath,file), log_file)
        else:
            print("BAD FILE:", file)
#         print('EXPERIMENT:\t',experiment_name)
#         print('SEED:\t\t', seed)
#         print('JOB ID:\t\t', jobid )
#         print('FRAMES:\t\t', frames)


In [5]:
job_id_table = pd.Series(job_id_dict)
job_id_table = job_id_table.sort_values()
job_id_table = job_id_table.unstack()

In [6]:
solution_table = pd.Series(solution_dict)
solution_table = solution_table.sort_values()
solution_table = solution_table.unstack()

In [7]:
solution_table.to_excel('solution.xlsx')
job_id_table.to_excel('jobid.xlsx')
solution_table.to_csv('solution.csv')
job_id_table.to_csv('jobid.csv')